In [ ]:
from glob import glob 
from fastcore.helpers import load_pickle
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
plt.style.use(['science', 'nature'])

In [ ]:
all_results = glob('results/20220919_polymer_inverse/*.pkl')

In [ ]:
all_results

['results/20220919_polymer_inverse/2022-09-19-17-29-13_results.pkl',
 'results/20220919_polymer_inverse/2022-09-19-17-01-14_results.pkl']

In [ ]:
d = load_pickle(all_results[0])

In [ ]:
d['metrics'][0][1].columns

Index(['distances', 'min', 'max', 'mean', 'expected_len', 'found_len',
       'monomer_squence', 'composition', 'smiles', 'prediction',
       'Levenshtein_min', 'Levenshtein_max', 'Levenshtein_mean',
       'Levenshtein_std', 'NormalizedLevenshtein_min',
       'NormalizedLevenshtein_max', 'NormalizedLevenshtein_mean',
       'NormalizedLevenshtein_std', 'LongestCommonSubsequence_min',
       'LongestCommonSubsequence_max', 'LongestCommonSubsequence_mean',
       'LongestCommonSubsequence_std', 'loss'],
      dtype='object')

In [ ]:
def summarize_at_temperature(subd): 
    mean_loss = np.mean(subd[0])
    mean_composition_mismatch = subd[1]['mean'].mean()
    mean_min_norm_lev = subd[1]['NormalizedLevenshtein_min'].min()
    mean_mean_norm_lev = subd[1]['NormalizedLevenshtein_mean'].mean()
    longest_common_subs_max_mean = subd[1]['LongestCommonSubsequence_max'].mean()
    longest_common_subs_mean_mean = subd[1]['LongestCommonSubsequence_mean'].mean()
    
    return {
        'mean_loss': mean_loss, 
        'mean_composition_mismatch': mean_composition_mismatch, 
        'mean_min_norm_lev': mean_min_norm_lev,
        'mean_mean_norm_lev': mean_mean_norm_lev, 
        'longest_common_subs_max_mean': longest_common_subs_max_mean
    }

In [ ]:
combined_results = []
optimal_results = []
for res in all_results: 
    try:
        d = load_pickle(res) 
        for t, td in d['metrics'].items():
            td_sum = summarize_at_temperature(td)
            td_sum['temperature'] = t

            combined_results.append(td_sum)


        td_sum = summarize_at_temperature(d['optimal_metrics'])

        optimal_results.append(td_sum)
    except Exception:
        pass

In [ ]:
combined_results = pd.DataFrame(combined_results)

In [ ]:
temperature & maximum common subsequence & composition mismatch & MAE 

In [ ]:
combined_results[['temperature', 'longest_common_subs_max_mean', 'mean_composition_mismatch', 'mean_loss']].round(2)

,temperature,longest_common_subs_max_mean,mean_composition_mismatch,mean_loss
0,0.00,58.55,4.26,3.07
1,0.25,59.63,4.26,3.10
2,0.50,60.26,4.26,3.04
3,0.75,60.91,4.26,3.06
4,1.00,61.24,4.28,3.09
5,1.25,62.09,4.29,3.08
6,1.50,65.87,3.85,3.01


In [ ]:
round